Import the necessary libraries.

In [1]:
# Import libraries for keywords
from transformers import BertModel, BertConfig, BertTokenizer, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, LineByLineTextDataset, Trainer, TrainingArguments
import numpy as np

Create the pre-trained BERT model.

In [2]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Get embeddings for text and print out their similarity.

In [3]:
# clusteredText = 'Cytosine: pyrimidine Correct _ This is a pyrimidine.. Thymine: pyrimidine Correct _ Thymine is a pyrimidine.. Uracil: pyrimidine Correct _ Uracil is a pyrimidine..'
# clusteredText = 'A structured and active process used to test a hypothesis _ In an experiment, the scientist has some form of control over the variables at work in the system under study _ Typically some variable is deliberately changed by the experimenter and other variables are monitored to see if they change as a result.. A structured and active process used to test a hypothesis _ In an experiment, the scientist has some form of control over the variables at work in the system under study _ Typically some variable is deliberately changed by the experimenter and other variables are monitored to see if they change as a result..  '
clusteredText = 'A group of similar cells carrying out the same detailed function within an organism _ In the lung, for instance, the lining of airways comprises one type of tissue: a layer of similar cells that forms a sealed surface _ . A groups of similar cells carrying out the same detailed function within an organism _ In the lung, for instance, the lining of airways comprises one type of tissue: a layer of similar cells that forms a sealed surface _ . '
clusteredText2 = 'Given that Mendel’s cross between these two true-breeding plants resulted in an F1 generation that all produced yellow round seeds, which of the following statements is true? Yellow and round are dominant traits; green and wrinkled are recessive traits _ Correct _ The offspring all have round yellow seeds, which means both these traits must be coded for by dominant alleles _ The parent phenotypes are “yellow round” and “green wrinkled.” All the offspring from these parents are “yellow round.”.  yellow,round 9 Correct _ Y (yellow) and R (round) are the dominant alleles _ Therefore any individual with at least one Y will be yellow and any individual with at least one R will be round _ Y (yellow) and R (round) are both dominant alleles _ Any individual with at least one Y will be yellow in color and any individual with at least one R will be round..  yellow, wrinkled 3 Correct _ Y (yellow) is the dominant allele and r (wrinkled) is the recessive allele _ Therefore any individual with at least one Y will be yellow, but an individual must have two r\'s (rr) to be wrinkled _ Y (yellow) is a dominant allele and r (wrinkled) is a recessive allele _ If an individual has at least one Y it will be yellow in color _ An individual has to have two r\'s (rr) to be wrinkled..  green, round 3 Correct _ y (green) is the recessive allele and R (round) is the dominant allele _ Therefore an individual must have two y\'s (yy) to be green and it will be round with at least one R allele _ y (green) is a recessive allele and R (round) is a dominant allele _ An individual must have two y\'s (yy) to be green in color _ An individual having at least on R will be round..  green, wrinkled 1 Correct _ Both y (green) and r (wrinkled) are recessive alleles _ Therefore an individual must have two y\'s (yy) to be green and two r\'s (rr) to be wrinkled _ Both y (green) and r (wrinkled) are recessive alleles _ An individual must have two y\'s (yy) to be green _ An individual must have two r\'s (rr) to be wrinkled..  '

# clusteredText = 'Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page..'
# clusteredText = 'Upon completing this module, students will be able to: . After examing atoms and their properties, students will be able to:. By the end of this exploration of water, students will be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. Upon completion of this module, students will be able to:. Upon completion of this module, students will be able to:. Upon investigation of transport proteins, students will be able to:. After studying this introduction to endocytosis, students will be able to:. Upon further exploratiion of endocytosis, students will be able to: .'
# clusteredText = 'You can take this quiz only once _ .  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once.. '

# clusteredText = ' Does a virus meet the definition of a cell? No Correct _ Although viruses do have a boundary between themselves and the environment, this boundary is a protein layer for many viruses _ For those viruses with a membrane, they lack other features found inside cells _ Therefore, they do not qualify as cells _ Some viruses are surrounded by a membrane structure, but not all of them are _ Viruses are particles that exist inside a living cell _ Viruses do not contain the machinery for growth and replication _ They must use the host cell’s machinery for their own replication _ A virus can reproduce only when the DNA is replicated inside of the cell of a living organism _ . You started life as a single cell, and took on your current form through a process of development and growth _ Why do cells in different parts of your body have different functions and characteristics? Each type of cell uses its DNA differently _ The DNA is similar, but DNA reacts to conditions and makes different products in each kind of cell _ Correct _ Through development, different cells come to behave differently _ They use different parts of the common “recipe book.” As a result, they produce specific proteins that allow them to carry out a specialized function _ Every time a cell divides, its DNA is copied and each of the resulting cells gets one full copy of the DNA from the original cell _ If two kitchens have identical libraries of recipe books, how might they still make very different meals? With very few exceptions, the cells in your body have similar DNA in their nuclei _ Each cell in your body has a similar set of “recipes,” but most of the recipes are not used _ In each specialized cell type, only a subset of products are made, and these give the cell its unique functions.. Which of the following is/are found in eukaryotic cell membranes? Select the single best option _ all of the above Correct.. If a cell is placed in an isotonic solution, which of the following will occur? none of the above Correct.. When a cell is placed in an unknown solution, the cell rapidly swells and bursts _ Which of the following would be true? The external solution is hypotonic relative to the cytoplasm of the cell _ Correct..  A zygote grows into an organism through what process? Mitosis Correct _ A zygote is the cell formed when a sperm fertilizes an egg _ Zygotes grow into adult through countless rounds of mitosis, which produces two identical daughter cells from a single cell _ A zygote is formed when egg and sperm meet _ In order for a zygote to grow, two identical daughter cells must be produced from a single cell _ Zygotes grow through mitosis.. A skin cell is an example of: a somatic cell Correct _ A somatic cell is a nonreproductive cell in a multicellular organism.. During the S phase of the cell cycle, DNA is replicated _ What form would you expect the DNA to be in? Unwound Correct _ DNA in chromatin form is easiest to replicate..  During the M (mitosis) phase of the cell cycle, what form would you expect the DNA to be in? Tightly wound around the histone proteins _ Correct _ In this form the chromosomes are compact and easy to move as the new cells are forming _ . Interphase is the part of the cell cycle when the cell: _ grows and duplicates its DNA _ Correct _ The cells grows in G1, duplicates its DNA in S, and makes final preparations in G2 _ DNA has to be copied (or replicated) before cell division occurs _ Cell division takes place during M _ In which phase is the cell growing and preparing to divide?.  In which stage of the cell cycle are new daughter cells formed? Cytokinesis Correct _ During cytokinesis the cytoplasm and membrane of the parental cell is split in order to form two daughter cells _ . In which stage of the cell cycle would you expect the growing of the cell to occur? G1 Correct _ During G1, the cell is growing (amassing nutrients and making the preparations for the remainder of the cell cycle) and the chromosomes condense in preparation for DNA synthesis and mitosis _ . In which stage of the cell cycle would checking for DNA integrity occur? G2 Correct _ During G2, the integrity of the newly replicated DNA is checked _ If there were errors in the replication of the DNA, the new daughter cells could be defective and harmful to the organism _ Therefore, the quality of the DNA is inspected during the G2 phase to ensure that each daughter cell is getting a good copy of the DNA _ . A muscle cell is an example of what type of cell? a somatic cell Correct _ A somatic cell is a non-reproductive cell in a multicellular organism.. DNA that is tightly wound around histone proteins is the easiest to replicate _ Is this statement true or false? False Yes _ Tightly wound DNA is not easy to replicate.. DNA that is tightly wound around histone proteins is the easiest to divide _ Is this statement true or false? True Yes _ When the DNA is tightly wound around histone proteins, it is easy to make sure one copy of each chromosome gets into the daughter cells.. During the S phase of the cell cycle, DNA is replicated _ What form would you expect the DNA to be in? It would be in a loose pile _ Yes! DNA in chromatin form is easiest to replicate.. During which stage of the cell cycle is the DNA replicated? S Yes _ This is a stage in interphase when the DNA is replicated.. During what stage of the cell cycle does the cell grow and carry out it regular functions? G1 Yes! G1 is the stage in which the cell carries out its daily functions.. If a cell is not dividing, and has not yet begun replicating its DNA what phase of the cell cycle would it be in? G1 Correct _ G1 occurs before DNA replication (S).. If you found a single cell with two nuclei, what explanation might be true? Mitosis occurred without cytokinesis! Yes _ If mitosis occurs, a cell will have two nuclei _ Cytokinesis must occur before there are two new cells.. All of the following pairings between a cell cycle phase and what happens during it are correct, except S- the cell is dividing Correct! Before the cell divides it has to copy its DNA, which happens during the S-phase of the interphase.. What role does mitosis play in the overall cell cycle? Mitosis is the process of dividing a parent cell into two genetically identical daughter cells _ DNA in the nucleus, which contains the genetic information, is first duplicated and condensed into tightly packed thread-like structures called chromosomes _ During the four phases of mitosis, the nucleus is separated and divided into two identical daughter nuclei _ Each daughter nuclei contains the same genetic material as the parental cell _ .  After completing the above activity, answer the following questions _ The inner circle in the above diagram is what part of the cell? Think about what function(s) that part perform(s) _ nucleus Correct _ The nucleus protects the chromosomes; it must be dismantled for mitosis to proceed _ The inner circle represents the control center of the cell..  At the end of the S stage, how many chromosomes will exist in the cell? double the number Correct _ During the synthesis (S) stage, the DNA is replicated so that the nucleus can be divided during mitosis and result in two genetically identical cells _ Chromosomes contain DNA and protein _ During the synthesis (S) stage of the cell cycle, DNA is duplicated _ Double the amount of DNA results in a duplicate number of chromosomes.. Explain what happens to the cell during the prophase of cell division _ The DNA condenses and the nuclear membrane begins to breakdown _ .  How many pairs of chromosomes will be found in each daughter canine skin cell after mitosis occurs? 39 Correct _ Each daughter cell will have 39 pairs of chromosomes, just like the parental cell..  You are studying mitosis in dog skin cells _ You observe a cell under a microscope that contains two sets of 39 chromosome pairs located on opposite sides of the cell _ You also note that it looks like nuclear envelopes are beginning to form around the chromosomes at each side of the cell _ The cell you observed is in which stage of mitosis? Telophase Correct _ In telophase, the daughter chromosomes have been pulled to opposite poles of the cell (which happened in anaphase), and a new nuclear envelope forms around each group of chromosomes.. What do proto-oncogenes do? They encourage the process of cell division _ Correct _ Proto-oncogenes encourage cell division.. Which function most closely describes how BOTH tumor suppressor genes and proto-oncogenes work? They regulate the cell cycle _ Correct _ While both genes have different functions, they both affect cell division by regulating the cell cycle.. An organism grows through what process? Mitosis Yes! Mitosis is cell division that results in growth..  Cancer can be the result of all of the following EXCEPT? A functional tumor suppressor gene and a functional proto-oncogene _ Correct _ Proto-oncogenes give the cell a signal to undergo cell division but only when the cell should be dividing _ Tumor suppressor genes, inhibit cell division so that a cell cannot divide uncontrollably _ It is when only these genes are both non-functional, that they can result in cancer development.. What is different about the second cell? In the first cell, the chromosomes are all one color _ In the second cell, the chromosomes have more than one color _ Correct _ In the second cell, crossing over occurred and the homologous chromosomes swapped DNA _ Look at the color of the chromosomes.. Prior to meiosis, the cell must go through the “S” stage of the cell cycle _ What happens during this stage? DNA is copied _ Correct _ The DNA is copied in the S stage of the cell cycle _ “S” stands for "synthesis," which means building _ Prior to dividing the chromosomes, they must be replicated.. Does the DNA in the skin cell have the same backbone as DNA in the brain cell of the same organism? Yes, all backbones of ALL DNA molecules are the same _ Correct! Since backbone is made of sugar and phosphate - there is no room for variation, it is always the same.. What process is responsible creating distinct sets of proteins in skin cell compared to brain cell? Gene Expression Correct. Does the DNA in the skin cell have the same sequence of bases as the DNA in the brain cell of the same organism? Yes, the sequence of bases should be the same in all cells of an organism _ Correct: Yes! We start out as a single cell and make a copy of DNA at each cell division, giving rise to different cell types _ Notice the word “should”…mutations do occur when DNA is not copied perfectly and some cells will have small changes in DNA sequence.. When a cell is getting ready to divide into two cells (during the process of mitosis), which process must first happen so that the entire genome can be passed to the new daughter cells? DNA replication Correct _ DNA replication produces a new double helix _ Each of these DNA molecules can then be passed on to the daughter cells..  Linear DNA: Eukaryotic only Eukaryotes have chromosomes made of linear DNA molecules..  DNA in nucleus: Eukaryotic only Only eukaryotic cells have a nucleus..  Circular DNA: Prokaryotic only Correct _ Prokaryotic cells have circular chromosomes and plasmids _ The nuclear genome of eukaryotic cells is composed of linear chromosomes..  DNA is wrapped tightly around histone proteins: Eukaryotic only Eukaryotic cells pack their long DNA molecules into the nucleus by winding DNA around proteins.. DNA resides in the nucleus of the cell in a prokaryotic cell, but not a eukaryotic cell _ False Correct.. DNA is circular in which type of cell? Both prokaryotic and eukaryotic cells _ Correct _ DNA is circular in both prokaryotic and eukaryotic cells.. In what type of cell do genes contain exons which are interrupted by introns, that are spliced out after transcription _ Eukaryotic Correct.. At what point in the cell cycle will eukaryotic DNA be most condensed (tightly packaged)? M phase Correct! During mitosis, when chromosomes need to be partitioned between two daughter cells, DNA is tightly packed into distinct chromosomes.. Eukaryotic DNA is packaged into nucleosomes due to the action of histones Correct.. '
# clusteredText = ' The human body contains bacterial cells and human cells _ In fact, bacterial cells outnumber human cells 10 to one _ Both are considered cells because they: can divide into new cells Correct _ The ability to divide into new cells is one of the tenets of cell theory..  Eggs and sperm are what type of cells? Gametes Correct _ Eggs and sperm are the reproductive cells of an organism that are produced through meiosis from germ cells _ Eggs and sperm join together through fertilization to produce a zygote _ Eggs and sperm are involved in sexual reproduction _ Eggs and Sperm of formed through meiosis..  Gametes are produced from what type of cell? Germ Correct _ Germ cells undergo meiosis to produce gametes (eggs and sperm) _ Germ cells are the progenator cells for eggs and sperm found in the ovaries and testes, respectively _ Gametes are formed through meiosis _ Eggs and Sperm are gametes _ .  Which of the following best describes a germ cell? A reproductive cell that undergoes meiosis to form gametes (eggs and sperm) _ Correct _ Germ cells are found in the ovaries and testes and produce eggs and sperm, respectively, through meiosis _ Germ cells do not refer to “germs” or infectious microorganisms _ Gametes are produced from germ cells _ Germ cells undergo meiosis. A skin cell would divide to make more skin cells using the process of: mitosis Correct _ Somatic cells like skin cells divide using mitosis.. Why must somatic cells be produced through mitosis? To ensure that old, damaged, or growing cells are replaced with cells that are of the same type and can perform the same function _ Correct _ Mitosis produces two daughter cells that are identical to the parental cell _ This ensures the integrity of the cells genetic make-up, function, and type from one generation of cells to the next _ . Why must gametes be produced through meiosis? Because gametes are used for sexual reproduction _ Correct _ Meiosis produces 4 haploid, genetically distinct daughter cells called gametes _ This ensures genetic diversity within a species through sexual reproduction _ . How many total chromosomes are found in one human zygote? 46 Correct _ A zygote, with 46 chromosomes, goes through mitosis to produce somatic cells, each with 46 chromosomes _ A zygote is the first cell of a new organism formed when a sperm fertilizes an egg _ Zygotes go through the process of mitosis to produce adult organisms _ A human zygote has the same number of chromosomes as the somatic cells in an adult human _ An adult human has 46 chromosomes.. Now take a wild guess: Remember that two gametes combine during fertilization to produce a single zygote _ How many chromosomes do you think are in one human gamete? 23 Correct _ Two human gametes, each with 23 chromosomes, will combine to form a zygote with 46 chromosomes _ Two gametes must combine to form a single zygote _ A human zygote has 46 chromosomes _ Two gametes combine to form a cell with 46 chromosomes _ So, how many chromosomes must be in each gamete?.  How many chromosomes are in the corn 2N cells? 20 Correct _ 2N is the total number of chromosomes in a diploid cell _ 2N is the total number of chromosomes in a diploid cell _ Somatic cells and germ cells are 2N _ Use the chart to determine how many chromosomes are in a somatic corn cell..  How many chromosomes are in a dog’s heart cells? 78 Correct _ The diploid chromosome number is the number of chromosomes in somatic cells _ Heart cells are somatic cells _ Somatic cells are diploid _ Use the chart to determine how many chromosomes are in a somatic dog cell..  How many chromosomes are in an unfertilized goldfish egg? 52 Correct _ The haploid number is in the column marked “chromosomes in gamete cells.” Unfertilized egg cells are gametes _ Gametes are haploid _ Use the chart to determine how many chromosomes are in a haploid goldfish cell..  How many chromosomes are in the cell that gave rise to the goldfish egg? 104 Correct _ This is the diploid number _ Unfertilized eggs are gametes and must be haploid, but the cell that gives rise to the gametes is diploid _ Germ cells give rise to gametes _ Gametes are haploid, but germ cells are diploid _ Use the chart to determine how many chromosomes are in a diploid goldfish cell..  Somatic corn cells have 20 chromosomes _ This means that in corn, _ _ 2n = 20 Correct _ 2n is the total number of chromosomes in a diploid, somatic cell _ 2n is the total number of chromosomes in a diploid cell _ Somatic cells are diploid _ In a diploid, somatic corn cell, there are 20 chromosomes..  In a dog, 2n = 78 _ How many chromosomes are in a dog’s heart cells? 78 Correct _ Dog heart cells are somatic, so they have 78 chromosomes _ Heart cells are somatic cells _ Somatic cells are diploid..  In goldfish, 2n = 104 _ How many chromosomes are in a goldfish egg? 52 Correct _ In diploid goldfish cells, there are 104 chromosomes _ There are half this many chromosomes (or 52) in haploid cells _ Egg cells are gametes _ Gametes are haploid _ Use the chart to determine how many chromosomes are in a haploid goldfish cell..  Remember that in goldfish, 2n = 104 _ How many chromosomes are in a goldfish germ cell? 104 Correct _ This is the diploid number _ Eggs are gametes and must be haploid, but the cell that gives rise to the gametes is diploid _ Germ cells give rise to gametes _ Gametes are haploid, but germ cells are diploid _ Diploid goldfish cells are 2n = 104.. Gametes are: haploid Correct _ A gamete is haploid, with only one copy of each chromosome.]. Somatic cells are reproduced during the process of _: mitosis Correct _ Somatic cells make copies of themselves through mitosis.. The products of MEIOSIS are: Four genetically unique cells Yes! Meiosis results in 4 unique cells.. The products of MITOSIS are: Two genetically identical cells Yes! Mitosis results in two genetically identical cells.. If a somatic cell has 24 chromosomes, how many chromosomes will you find in the daughter cells, after mitosis? 24 Yes! Somatic cells divide by mitosis, which results in 2 identical daughter cells.. If a germ cell has 46 chromosomes, how many chromosomes will you find in the daughter cells, after meiosis? 23 Yes _ Germ cells divide by meiosis, which results in 4 unique daughter cells, each with half the original number of chromosomes.. If a germ cell has 20 chromosomes, how many chromosomes will you find in the daughter cells, after meiosis? 10 Yes _ Germ cells divide by meiosis, which results in 4 unique daughter cells, each with half the original number of chromosomes.. Imagine a hypothetical organism (critter) _ A DIPLOID cell from this organism has 20 chromosomes _ How many chromosomes would be found in a HAPLOID GAMETE of this organism? 10 Yes! A haploid cell has half the number of chromosomes found in a diploid cell.. Imagine a hypothetical organism (critter) _ A DIPLOID cell from this organism has 20 chromosomes _ How many chromosomes would be found in a GERM CELL of this organism? 20 Yes _ This is the number of chromosomes in the diploid cell _ Germ cells are diploid.. How many cells are there at the end of telophase? One Correct _ The cell now has two nuclei, but they have not been split into separate cells yet..  True or False? A dog skin cell containing 78 chromosomes is a diploid cell _ True Correct _ Dog skin cells are somatic cells and contain two copies of each of the 39 chromosomes _ This makes them diploid _ Dog gamete cells (eggs or sperm) are haploid and contain only one copy of each chromosome.. Which of the following tasks is accomplished through the process of mitosis? Replacement of dysfunctional cells Yes _ Mitosis enables dysfunctional cells to be replaced.. What is the name of the process of asexual cell division that results in daughter cells that are identical to the parent cell? Mitosis Correct _ Mitosis is the cell division that results in two identical daughter cells _ Cell division is part of the cell cycle _ Somatic cells and germ cells reproduce asexually _ Somatic cells undergo mitosis.. What is the name of the process of sexual cell division that results in daughter cells that are NOT identical to their parent cell? Meiosis Correct _ Meiosis does not result in identical daughter cells _ Cell division is part of the cell cycle _ Somatic cells and germ cells reproduce asexually _ Somatic cells undergo mitosis..  A hypothetical cell has a chromosome number of 2n = 16 _ Which of the following statements is TRUE? The diploid cells from this species have 8 homologous pairs of chromosomes _ Correct _ The cell has 2n = 16 chromosomes, so n = 8 _ This means the cell has 8 different chromosomes, but has 2 copies of each _ “N” refers to the number of chromosome sets in a cell _ In diploid cells (2n), chromosomes are in pairs.. How many chromosomes are there in each cell AFTER meiosis I? 2 Correct _ If you count the centromeres, you’ll see that now there are two chromosomes in each cell _ The number of chromosomes in the cell was cut in half _ This is why meiosis I is called the “reduction division.” The cell on the left is before meiosis I _ The 2 cells on the right are at the end of meiosis I, and about to enter meiosis II _ Count the number of centromeres in each cell to determine the number of chromosomes _ The cells on the right each have two centromeres.. Meiosis is important for sexual reproduction in all of the following ways except? Maintaining proper function of the germ cells Correct _ Germs cells produce gametes (egg and sperm) through meiosis _ Meiosis does not maintain the function of the germs cells _ Maintaining the proper functioning of germ cells is the job of hormones..  At the end of telophase II, the number of chromosomes is 1N (haploid) Correct _ At the end of meiosis, the daughter cells are haploid and contain one copy of each chromosome _ The parental cell was diploid and had two copies of each chromosome..  Meiosis and sexual reproduction produce genetic variability in all of the following ways except? Producing identical daughter cells Correct _ Identical daughter cells are produced in mitosis and since they have identical genes and chromosomes, this does not increase genetic diversity.. What is the purpose of meiosis? To enable an organism to produce gametes Yes _ Meiosis results in haploid gametes for sexual reproduction. Which of the following tasks is accomplished through the process of meiosis? Production of sperm or eggs Yes _ Meiosis results in sperm or egg production.. Mitosis results in _ cells and meiosis results in _ cells _ diploid, haploid Yes _ Mitosis results in diploid cells and meiosis results in haploid cells.. Mitosis results in _ and meiosis results in _ _ somatic cells, gametes Yes _ Mitosis results in somatic cells and meiosis results in gametes.. Corn has 20 chromosomes _ When corn undergoes meiosis how many chromosomes are found in the gametes? 10 Correct _ Meiosis results in half the number of chromosomes.. Cats have a diploid number of 38 _ How many chromosomes are in a cat egg? 19 Correct _ Haploid cells have half the number of chromosomes as diploid cells.. How many cells are present after meiosis I? 2 Correct _ Meiosis I produces two cells _ Meiosis has two stages; this question is asking how many cells are present after meiosis I only _ Meiosis I is only halfway through the process of meiosis and would have produced half the total number of cells _ Although meiosis results in four cells, meiosis I is only halfway through the process and has only produced half the total number of cells.. After meiosis I, these cells will enter meiosis II _ How many cells are present after meiosis II? 4 Correct _ Each of the two cells divides again to produce four total cells _ Each of the cells from meiosis I divides again in meiosis II _ Meiosis I ends with two cells, each of which divides again.. How many gametes does a diploid cell make during meiosis? four haploid cells Correct _ Meiosis produces four haploid cells _ Meiosis produces cells with half the number of chromosomes as the original cell _ The original cell was diploid _ Meiosis produces four cells.. What percentage of the gametes produced will have the “A” allele? 50% Correct _ Two of the four cells have this allele _ Review the images above; how many of the cells have an “A”? Of the four cells above, two have an “A,” and two have an “a.”. What percentage of the gametes produced will have the “a” allele? 50% Correct _ Two of the four cells have this allele _ Review the images above; how many of the cells have an “a”? Of the four cells above, two have an “A,” and two have an “a.”. What is the name of the type of cell division that gives rise to gametes (reproductive cells)? meiosis Correct _ Meiosis results in haploid gametes (reproductive cells).. Which of the following is true of a diploid organism? All of the above _ Correct _ A diploid organism has two chromosomes, two genes, and each gene has two alleles.. A haploid organism has how many copies of each chromosome? 1 Correct _ A haploid organism has one of each chromosome.. How many copies of each gene does a diploid organism have? 2 Correct _ A diploid organism has two of each chromosome, and therefore two of each gene.. The cells of your body are surrounded by water _ The outer layer of these cells is made of many nonpolar molecules _ What would happen if the outer layer of these cells was made of mainly polar molecules? The cells would dissolve _ Correct _ The nonpolar characteristic of this layer allows cells to survive in an aqueous environment.. Skin cells perform very different functions compared to brain cells _ On the molecular level, what makes skin cells different from brain cells in the same organism? Proteins Correct. '
texts = [clusteredText, clusteredText2]

max_length = 512

# train_encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path = 'raw-text-for-bert.txt',
    block_size = 512
    )
print('No. of lines: ', len(dataset))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir='/continue_pretrain',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
    # prediction_loss_only=True
)

trainer.train()
trainer.save_model('continue_pretrain/test_model')

/home/jwood/environments/SMART-env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


No. of lines:  2703


Step,Training Loss
500,1.840500
1000,1.447200
1500,1.334600
2000,1.256900
2500,1.207700
3000,1.169300
3500,1.086600
4000,1.020200
4500,1.002300
5000,0.997000
